# Dataframe basics
### Dr. Tirthajyoti Sarkar, Fremont, CA 94536
### Apache Spark
Apache Spark is one of the hottest new trends in the technology domain. It is the framework with probably the highest potential to realize the fruit of the marriage between Big Data and Machine Learning. It runs fast (up to 100x faster than traditional Hadoop MapReduce) due to in-memory operation, offers robust, distributed, fault-tolerant data objects (called RDD), and inte-grates beautifully with the world of machine learning and graph analytics through supplementary
packages like Mlib and GraphX.

Spark is implemented on Hadoop/HDFS and written mostly in Scala, a functional programming language, similar to Java. In fact, Scala needs the latest Java installation on your system and runs on JVM. However, for most of the beginners, Scala is not a language that they learn first to venture into the world of data science. Fortunately, Spark provides a wonderful Python integration, called PySpark, which lets Python programmers to interface with the Spark framework and
learn how to manipulate data at scale and work with objects and algorithms over a distributed file system.

### Dataframe
In Apache Spark, a DataFrame is a distributed collection of rows under named columns. It is conceptually equivalent to a table in a relational database, an Excel sheet with Column headers, or a data frame in R/Python, but with richer optimizations under the hood. DataFrames can be constructed from a wide array of sources such as: structured data files, tables in Hive, external databases, or existing RDDs. It also shares some common characteristics with RDD:

* Immutable in nature : We can create DataFrame / RDD once but can’t change it. And we can transform a DataFrame / RDD after applying transformations.
* Lazy Evaluations: Which means that a task is not executed until an action is performed.
* Distributed: RDD and DataFrame both are distributed in nature.

### Advantages of the DataFrame
* DataFrames are designed for processing large collection of structured or semi-structured data.
* Observations in Spark DataFrame are organised under named columns, which helps Apache Spark to understand the schema of a DataFrame. This helps Spark optimize execution plan on these queries.
* DataFrame in Apache Spark has the ability to handle petabytes of data.
* DataFrame has a support for wide range of data format and sources.
* It has API support for different languages like Python, R, Scala, Java.

In [1]:
import pyspark
from pyspark import SparkContext as sc

### Create a `SparkSession` app object

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark1 = SparkSession.builder.appName('Basics').getOrCreate()

### Read in a JSON file and examine the data

In [4]:
df = spark1.read.json('Data/people.json')

#### Unlike Pandas DataFrame, it does not show itself when called

Instead, it just shows the Data types of the columns

In [5]:
df

DataFrame[age: bigint, name: string]

#### You have to call show() method to evaluate it i.e. show it

In [6]:
df.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



### The data schema
Use `printSchema()` to show he schema of the data. Note, how tightly it is integrated to the SQL-like framework. You can even see that the schema accepts `null` values because nullable property is set `True`.

In [7]:
df.printSchema()

root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)



#### Fortunately a simple `columns` method exists to get column names back as a Python list

In [8]:
df.columns

['age', 'name']

### The `describe` and `summary` methods

Similar to Pandas, the `describe` method is used for the statistical summary. But unlike Pandas, calling only `describe()` returns a DataFrame! This is due to the **[lazy evaluation](https://data-flair.training/blogs/apache-spark-lazy-evaluation/)** - the actual computation is delayed as much as possible.

In [9]:
df.describe()

DataFrame[summary: string, age: string, name: string]

#### We have to call `show` again

In [10]:
df.describe().show()

+-------+------------------+-------+
|summary|               age|   name|
+-------+------------------+-------+
|  count|                 2|      3|
|   mean|              24.5|   null|
| stddev|7.7781745930520225|   null|
|    min|                19|   Andy|
|    max|                30|Michael|
+-------+------------------+-------+



#### There is `summary` method for more stats

In [11]:
df.summary().show()

+-------+------------------+-------+
|summary|               age|   name|
+-------+------------------+-------+
|  count|                 2|      3|
|   mean|              24.5|   null|
| stddev|7.7781745930520225|   null|
|    min|                19|   Andy|
|    25%|                19|   null|
|    50%|                19|   null|
|    75%|                30|   null|
|    max|                30|Michael|
+-------+------------------+-------+



### The `take` and `collect` methods to read/collect rows

These methods return some or all rows as a Python list.

In [12]:
df.take(2)

[Row(age=None, name='Michael'), Row(age=30, name='Andy')]

In [13]:
df.collect()

[Row(age=None, name='Michael'),
 Row(age=30, name='Andy'),
 Row(age=19, name='Justin')]

### Defining your own Data Schema

Import data types and structure types to build the data schema yourself

In [14]:
from pyspark.sql.types import StructField, IntegerType, StringType, StructType

Define your data schema by supplying name and data types to the structure fields you will be importing. It will be a simple Python list of `StructField` objects. You have to use Spark data types like `IntegerType` and `StringType`.

In [15]:
data_schema = [StructField('age',IntegerType(),True),
StructField('name',StringType(),True)]

Now create a `StrucType` object called `final_struc` with this schema as field

In [16]:
final_struc = StructType(fields=data_schema)

Now read in the same old JSON with this new schema `final_struc`

In [17]:
df = spark1.read.json('Data/people.json',schema=final_struc)
df.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



Now when you print the schema, **you will see that the `age` is read as `int` and not `long`**. By default Spark could not figure out for this column the exact data type that you wanted, so it went with `long`. But this is how you can build your own schema and instruct Spark to read the data accoridngly.